# GPT Explainer

the purpose of this is to walk through what happens during the forward pass of GPT-2 like models.  To help display the transformation, we'll use the first sentence from the [linear algebra wiki page](https://en.wikipedia.org/wiki/Linear_algebra) and [lu decomposition wiki page](https://en.wikipedia.org/wiki/LU_decomposition) as the topic is fitting and it shows us some non-standard patterns.  

## Text Prep/Tokenization
we'll start with common preprocessing step of tokenizing the data.  This converts the string text into an array of numbers that can be used during the training loop.  I've built a very subtle byte-pair encdoing that has each unique character that appears and the top 5 merges. This keeps our vocab size small and managable for this example. Typically the vocab size is in the 100K+ range. A great library for this is `tiktoken`. Tokenization simply finds the longest pattern of characters that's in common with what was trained and replaces it with an integer that represents it.  This way we turn the text into a numeric array to simplify computing. 

In [1]:
import torch
from collections import Counter

In [2]:
class SimpleBPETokenizer:
    def __init__(self, num_merges=5, eot_token='<|endoftext|>'):
        self.num_merges = num_merges
        self.eot_token = eot_token
        self.eot_id = None
        self.merges = []
        self.pair_ranks = {}
        self.vocab = {}
        self.id_to_token = {}

    def _add_token(self, tok):
        if tok in self.vocab:
            return self.vocab[tok]
        i = len(self.vocab)
        self.vocab[tok] = i
        self.id_to_token[i] = tok
        return i

    def _get_bigrams(self, seq):
        for i in range(len(seq) - 1):
            yield (seq[i], seq[i + 1])

    def _merge_once(self, seq, pair):
        a, b = pair
        out = []
        i = 0
        while i < len(seq):
            if i < len(seq) - 1 and seq[i] == a and seq[i + 1] == b:
                out.append(a + b)
                i += 2
            else:
                out.append(seq[i])
                i += 1
        return out

    def train(self, corpus):
        # corpus: list[str]
        text = ''.join(corpus).lower()
        seq = list(text)
        merges = []
        for _ in range(self.num_merges):
            counts = Counter(self._get_bigrams(seq))
            if not counts: break
            best_pair, _ = counts.most_common(1)[0]
            merges.append(best_pair)
            seq = self._merge_once(seq, best_pair)
        self.merges = merges
        self.pair_ranks = {p: i for i, p in enumerate(self.merges)}

        self.vocab = {}
        self.id_to_token = {}
        for ch in sorted(set(text)):
            self._add_token(ch)
        for a, b in self.merges:
            self._add_token(a + b)
        self.eot_id = self._add_token(self.eot_token)

    def encode(self, text, force_last_eot=True):
        # treat literal eot marker as special; remove it from content
        if self.eot_token in text:
            text = text.replace(self.eot_token, '')
        seq = list(text)

        # make sure all seen base chars exist
        for ch in set(seq):
            if ch not in self.vocab:
                self._add_token(ch)

        # greedy BPE using learned pair ranks
        if self.merges:
            while True:
                best_pair, best_rank = None, None
                for p in self._get_bigrams(seq):
                    r = self.pair_ranks.get(p)
                    if r is not None and (best_rank is None or r < best_rank):
                        best_pair, best_rank = p, r
                if best_pair is None:
                    break
                seq = self._merge_once(seq, best_pair)

        # ensure all tokens in seq exist in vocab (e.g., if new chars appeared)
        for tok in seq:
            if tok not in self.vocab:
                self._add_token(tok)

        ids = [self.vocab[tok] for tok in seq]

        # FORCE: append EOT id if not already last
        if force_last_eot:
            if not ids or ids[-1] != self.eot_id:
                ids.append(self.eot_id)

        return ids

    def decode(self, ids):
        # drop trailing EOT if present
        if ids and self.eot_id is not None and ids[-1] == self.eot_id:
            ids = ids[:-1]
        toks = [self.id_to_token[i] for i in ids]
        return ''.join(toks)


In [3]:
raw_example_1 = r'''Linear algebra is central to almost all areas of mathematics. For instance, linear algebra is fundamental in modern presentations of geometry, including for defining basic objects such as lines, planes and rotations. Also, functional analysis, a branch of mathematical analysis, may be viewed as the application of linear algebra to function spaces.'''
raw_example_2 = r'''In numerical analysis and linear algebra, lower–upper (LU) decomposition or factorization factors a matrix as the product of a lower triangular matrix and an upper triangular matrix (see matrix multiplication and matrix decomposition).'''


In [4]:
tok = SimpleBPETokenizer(num_merges=5)
tok.train([raw_example_1,raw_example_2])
tok.merges

[(' ', 'a'), ('a', 't'), ('i', 'n'), (' ', 'm'), ('i', 'o')]

In [5]:
tok.vocab

{' ': 0,
 '(': 1,
 ')': 2,
 ',': 3,
 '.': 4,
 'a': 5,
 'b': 6,
 'c': 7,
 'd': 8,
 'e': 9,
 'f': 10,
 'g': 11,
 'h': 12,
 'i': 13,
 'j': 14,
 'l': 15,
 'm': 16,
 'n': 17,
 'o': 18,
 'p': 19,
 'r': 20,
 's': 21,
 't': 22,
 'u': 23,
 'v': 24,
 'w': 25,
 'x': 26,
 'y': 27,
 'z': 28,
 '–': 29,
 ' a': 30,
 'at': 31,
 'in': 32,
 ' m': 33,
 'io': 34,
 '<|endoftext|>': 35}

In [6]:
vocab_size = len(tok.vocab)
vocab_size

36

In [7]:
eot = tok.eot_id
tokens = []
for example in [raw_example_1, raw_example_2]:
    tokens.extend([eot])
    tokens.extend(tok.encode(example.lower()))
all_tokens = torch.tensor(tokens, dtype=torch.long)
all_tokens

tensor([35, 15, 32,  9,  5, 20, 30, 15, 11,  9,  6, 20,  5,  0, 13, 21,  0,  7,
         9, 17, 22, 20,  5, 15,  0, 22, 18, 30, 15, 16, 18, 21, 22, 30, 15, 15,
        30, 20,  9,  5, 21,  0, 18, 10, 33, 31, 12,  9, 16, 31, 13,  7, 21,  4,
         0, 10, 18, 20,  0, 32, 21, 22,  5, 17,  7,  9,  3,  0, 15, 32,  9,  5,
        20, 30, 15, 11,  9,  6, 20,  5,  0, 13, 21,  0, 10, 23, 17,  8,  5, 16,
         9, 17, 22,  5, 15,  0, 32, 33, 18,  8,  9, 20, 17,  0, 19, 20,  9, 21,
         9, 17, 22, 31, 34, 17, 21,  0, 18, 10,  0, 11,  9, 18, 16,  9, 22, 20,
        27,  3,  0, 32,  7, 15, 23,  8, 32, 11,  0, 10, 18, 20,  0,  8,  9, 10,
        32, 32, 11,  0,  6,  5, 21, 13,  7,  0, 18,  6, 14,  9,  7, 22, 21,  0,
        21, 23,  7, 12, 30, 21,  0, 15, 32,  9, 21,  3,  0, 19, 15,  5, 17,  9,
        21, 30, 17,  8,  0, 20, 18, 22, 31, 34, 17, 21,  4, 30, 15, 21, 18,  3,
         0, 10, 23, 17,  7, 22, 34, 17,  5, 15, 30, 17,  5, 15, 27, 21, 13, 21,
         3, 30,  0,  6, 20,  5, 17,  7, 

# Modeling
A machine learning model forward pass now uses the tokenization information, runs several layers of linear algebra on it, and then "predicts" the next token. When it is noisy (like you will see in this example), this process results in gibberish.  The training process changes the noise to pattern during the "backward pass" as you'll see.    We'll show 3 steps that are focused on training:
1. **Data Loading** `x, y = train_loader.next_batch()` - this step pulls from the raw data enough tokens to complete a forward and backward pass.  If the model is inference only, this step is replaced with taking in the inference input and preparing it similarly as the forward pass.
2. **Forward Pass** `logits, loss = model(x, y)` - using the data and the model architecture to predict the next token. When training we also compare against the expected to get loss, but in infrerence, we use the logits to complete the inference task.
3. **Backward Pass** `loss.backward()` - using differentials to understand what parameters most impact the forward pass' impact on its prediction, comparing that against what is actually right based on the data loading step, and then making very minor adjustments to the impactful parameters with the hope it improves future predictions.  

## Data Loading
To start, we need to get enough data to run the forward and backward passes.  Since our total dataset is likely too big to hold all at once in real practice, we would read just enough file information into memory so that we can run the passes, leaving memory and compute to be used on the passes instead of static data holding. 
To start, we have to identify the batch size and the model context length to determine how much data we need.  Consequently, these dimensions also form 2 of the 3 dimensions in the initial matrix.
- **Batch Size (B)** - This is the number of examples you'll train on in a single pass. 
- **Context Length (T)** - This is the max number of tokens that a model can use in a single pass to generat the next token. If an example is below this length, it can be padded.
  
*Ideally both B and T are multiples of 2 to work nicely with chip architecture. This is a common theme across the board*

In [8]:
B = 2 # Batch
T = 8 # context length

To start, we need to pull from our long raw_token list enough tokens for the forward pass. To be able to satisfy training `B` Batches `T` Context length, we need to pull out `B*T` tokens to slide the context window across the examples enough to satisfy the batch size.  Since the training will attempt to predict the last token given the previous tokens in context, we also need 1 more token at the end so that the last training example in the last batch can have the next token to validate against. 

In [9]:
current_position = 0
tok_for_training = all_tokens[current_position:current_position + B*T +1 ]
tok_for_training

tensor([35, 15, 32,  9,  5, 20, 30, 15, 11,  9,  6, 20,  5,  0, 13, 21,  0])

Now that we have our initial tokens to train on, we now need to convert it to a matrix that's ready for training. In this step we'll need to create our batches and setup two different arrays: 1/ the input, `x`, tokens that will result in 2/ the output `y` tokens. To create each example in the batch, every `T` tokens will be placed into it's own row. 

Recall that training takes in a string of tokens the length of the context and then predicts the next token. Recall that when we extracted `tok_for_training` we added 1 extra token so that we can evaluate the prediction for the last example. Because of this, the input, `x`, will be all of the tokens up to the second to last element `[:-1]`.  

It might be natural to think the output `y` would then just be the last token.But this is actually wasting valuable training loops.  Yes, there is the example that fills the context `T`, but we also have enough tokens in `tok_for_training` where any context length of `n` where `n<T` can also be used for inference since we have the `n+1` token available.  You can think of the following example:

sentence: `Hi I am learning`. This sentence contains the following "next tokens" that can be learned:
1. x: Hi I am  | y: learning
2. x: Hi I     | y: am
3. x: Hi       | y: I

Because we have this triangle to create, our `y` can be much larger.  We can start with the second token and, go all the way to the last element we added for the last example `[1:'`.   


We will now put this together and do the following:
1. Extract the input `x` and then split it into an example for each batch `B`
2. Extract the output `y` and then split it into an example for each batch `B`

*Note: View can take `-1` which allows the matrix to infer the dimension so we do not need to pass in `T`, but given how many matrices we'll work with we want to make sure we're controlling the dimensions or erroring out if they do not match our expectations.*

In [10]:
tok_for_training

tensor([35, 15, 32,  9,  5, 20, 30, 15, 11,  9,  6, 20,  5,  0, 13, 21,  0])

In [11]:
x=tok_for_training[:-1].view(B, T)
x

tensor([[35, 15, 32,  9,  5, 20, 30, 15],
        [11,  9,  6, 20,  5,  0, 13, 21]])

In [12]:
y=tok_for_training[1:].view(B, T)
y

tensor([[15, 32,  9,  5, 20, 30, 15, 11],
        [ 9,  6, 20,  5,  0, 13, 21,  0]])

## Forward pass

![image.png](explainer_screenshots/full_network.png)

The forward pass takes a string of tokens in and predicts the next "n" tokens.  This step as we'll look at it is focused on training where we'll pass in the input `x`, carry that input through the layers, and generate a matrix of the probability of each token being the next one, something we call `logits`. During the forward pass at the end we then compare the probability to the actual next token in `y` and calculate `loss` based on the difference. 

*Note that we will do some layer initialization to simplify following along.  In reality layers are often initialized to normal distribution with some adjustments made for parameter sizes to keep the weights properly noisy.  We will not cover initialization in this series*

We first rederive the batch size and context size based on the input to improve flexibility.

In [13]:
import torch.nn as nn

In [14]:
B, T = x.size()
B,T

(2, 8)

The first layer of our network creates an embedding representation of our input sequence. 

In [15]:
block_size = 8 # max sequence length/context
vocab_size = vocab_size # 36 
n_embd = 4 # 

### Input Layer

![image.png](explainer_screenshots/input_layer.png)

We'll first create an initiation for 2 of our input matrices: position and token embedding.  Both of these are a table of weigths that have `n_embd` number of columns to store information about the position or token. The more columns you add, the more complex information can be stored but the more compute is needed.  For now we'll let each position or column store up to 4 channels of information.  Before starting though we need to initialize the layer with a set of data.

In [16]:
# weighted token embedding
wte = nn.Embedding(vocab_size, n_embd)
with torch.no_grad(): # initilize to W[i,j] = 0.001*(1+i+j) for easy following 
    vs, d = wte.num_embeddings, wte.embedding_dim
    rows = torch.arange(vs).unsqueeze(1)  # (vs,1)
    cols = torch.arange(d).unsqueeze(0)  # (1,d)
    pattern = (1 + rows + cols)  # W[i,j] = 0.001*(1+i+j)
    wte.weight.copy_(pattern)
# weighted position embedding
wpe = nn.Embedding(block_size, n_embd)
with torch.no_grad(): # initilize to W[i,j] = 0.001*(1+i+j) for easy following 
    vs, d = wpe.num_embeddings, wpe.embedding_dim
    rows = torch.arange(vs).unsqueeze(1)  # (vs,1)
    cols = torch.arange(d).unsqueeze(0)  # (1,d)
    pattern = (1 + rows + cols)  # W[i,j] = 0.001*(1+i+j)
    wpe.weight.copy_(pattern)
wte.weight, wpe.weight

(Parameter containing:
 tensor([[ 1.,  2.,  3.,  4.],
         [ 2.,  3.,  4.,  5.],
         [ 3.,  4.,  5.,  6.],
         [ 4.,  5.,  6.,  7.],
         [ 5.,  6.,  7.,  8.],
         [ 6.,  7.,  8.,  9.],
         [ 7.,  8.,  9., 10.],
         [ 8.,  9., 10., 11.],
         [ 9., 10., 11., 12.],
         [10., 11., 12., 13.],
         [11., 12., 13., 14.],
         [12., 13., 14., 15.],
         [13., 14., 15., 16.],
         [14., 15., 16., 17.],
         [15., 16., 17., 18.],
         [16., 17., 18., 19.],
         [17., 18., 19., 20.],
         [18., 19., 20., 21.],
         [19., 20., 21., 22.],
         [20., 21., 22., 23.],
         [21., 22., 23., 24.],
         [22., 23., 24., 25.],
         [23., 24., 25., 26.],
         [24., 25., 26., 27.],
         [25., 26., 27., 28.],
         [26., 27., 28., 29.],
         [27., 28., 29., 30.],
         [28., 29., 30., 31.],
         [29., 30., 31., 32.],
         [30., 31., 32., 33.],
         [31., 32., 33., 34.],
         [32., 3

**Positional Embeddings** - Now we need to pluck the weight of each position out of the position embedding.  Since we are creating a simple left to right, position 1 to n, we can just create an array from 0 to n based on the context, `T`, then pluck those rows out.  The resulting matrix from this operation is a `T, n_embd` based vector. 

In [17]:
pos = torch.arange(0, T, dtype=torch.long)
pos

tensor([0, 1, 2, 3, 4, 5, 6, 7])

for each element, look up the row in `wpe` and pluck it out. Since the position is just `[0:T]` we can see we pluck out the position array.

In [18]:
pos_emb = wpe(pos)
pos_emb.shape, pos_emb

(torch.Size([8, 4]),
 tensor([[ 1.,  2.,  3.,  4.],
         [ 2.,  3.,  4.,  5.],
         [ 3.,  4.,  5.,  6.],
         [ 4.,  5.,  6.,  7.],
         [ 5.,  6.,  7.,  8.],
         [ 6.,  7.,  8.,  9.],
         [ 7.,  8.,  9., 10.],
         [ 8.,  9., 10., 11.]], grad_fn=<EmbeddingBackward0>))

**Word Embeddings** - Similarly we need to pluck out the rows from the token table, `wte` for the tokens in our example. Since our example is already represented as indices, we can simple use `x` directly. The resulting matrix from this operation is a `B,T, n_embd` based vector since `x` is `B,T` and `tok_emb` is `vocab_size,n_embd` and when we index `wte` by `x` each entry in x replaces `n_embd` based vector at that position in `wte`.

In [19]:
x

tensor([[35, 15, 32,  9,  5, 20, 30, 15],
        [11,  9,  6, 20,  5,  0, 13, 21]])

for each position pull out the the row of weights that corresponds to the token. You can see in the print out that the rows are not in the same order as the layer is initiliazed as the token ids are not sequential.  

In [20]:
tok_emb = wte(x)
tok_emb.shape, tok_emb

(torch.Size([2, 8, 4]),
 tensor([[[36., 37., 38., 39.],
          [16., 17., 18., 19.],
          [33., 34., 35., 36.],
          [10., 11., 12., 13.],
          [ 6.,  7.,  8.,  9.],
          [21., 22., 23., 24.],
          [31., 32., 33., 34.],
          [16., 17., 18., 19.]],
 
         [[12., 13., 14., 15.],
          [10., 11., 12., 13.],
          [ 7.,  8.,  9., 10.],
          [21., 22., 23., 24.],
          [ 6.,  7.,  8.,  9.],
          [ 1.,  2.,  3.,  4.],
          [14., 15., 16., 17.],
          [22., 23., 24., 25.]]], grad_fn=<EmbeddingBackward0>))

**Impact of the position and token together**
To ensure that the position and token together impact the next token prediction, we sum the two so that the weight of each token is impacted by the weight of its relative position. To do this we sum `tok_emb` and `pos_emb` together. Quickly we can see the dimensions don't match as 
* `tok_emb` > `B,T,n_embd`
* `pos_emb` >   `T,n_embd`

Since we have multiple examples with the same ordering, we simply add pos_emb at the same level to each entry on the `B` dimension, something that pytorch does for us automatically resulting in a `B,T,n_embd` output

In [21]:
x = tok_emb + pos_emb
x

tensor([[[37., 39., 41., 43.],
         [18., 20., 22., 24.],
         [36., 38., 40., 42.],
         [14., 16., 18., 20.],
         [11., 13., 15., 17.],
         [27., 29., 31., 33.],
         [38., 40., 42., 44.],
         [24., 26., 28., 30.]],

        [[13., 15., 17., 19.],
         [12., 14., 16., 18.],
         [10., 12., 14., 16.],
         [25., 27., 29., 31.],
         [11., 13., 15., 17.],
         [ 7.,  9., 11., 13.],
         [21., 23., 25., 27.],
         [30., 32., 34., 36.]]], grad_fn=<AddBackward0>)

### Transformer Layers

![image.png](explainer_screenshots/transformer.png)

The transformer block is multiple parallel repetitions of the same matrix operations done independently.  This adds both depth and breadth to the computation.  Each block is the same steps of
1. Layer normalization
2. Causal self attention
3. Layer normalization (again)
4. Multi-layer perceptron (MLP)

Both steps 2 and 4 are also multi-layered so we'll go through each layer independently.  You'll notice the arrows in the diagram bypassing the causal self attention and the MLP.  This is to ensure that the weights of any one layer do not get overweighted. We achieve this by simply adding the input with the layer's calculations together, as you'll see. 

#### Transformer - Layer Normalization
With Layer normalization, we review the row and adjust based on how far away it is from the mean. This means an array of `[1,2,3,4]` and `[2,4,6,8]` will actually have the same normalized entries after layer normalization.  This layer adds regularization which helps with overall learning speed. The formula applied is:

$y = \frac{x - \mathbb{E}[x]}{\sqrt{\operatorname{Var}[x] + \epsilon}}$

Layer normalization is applied on the input matrix in kind and creates default weights of 1's across the dimension to equally weight all values in the normalization. We'll keep this as is and not change the initialization.  

*Note that even though we will do layer normalization again, we keep this as a separate layer so that its impact can be adjusted independent of other normalization layers.*

**Example** Let's see a quick example of how layer normalizaiton operates with an array of `[1,2,3,4]` and `[2,4,6,8]` 

In [22]:
example_ln = nn.LayerNorm(n_embd)

## Example 
example_ln(torch.tensor(
    [
        [1.0,2.0,3.0,4.0],
        [2.0,4.0,6.0,8.0]
    ]))

tensor([[-1.3416, -0.4472,  0.4472,  1.3416],
        [-1.3416, -0.4472,  0.4472,  1.3416]],
       grad_fn=<NativeLayerNormBackward0>)

**Normalize** Now let's apply it to x.  We'll save the output to a new variable `x_norm` so that we retain the input `x` for the *skip connection* (more details below).  Since our position embeddings are still in their initiaion step where each entries values are similarly distributed `[n,n+2,n+4,n+6]`  we'll see that with layer normalization the resulting matrix has all equal rows. 

In [ ]:
ln_1 = nn.LayerNorm(n_embd)
x_norm = ln_1(x)
x_norm.shape, x_norm

#### Transformer - Causal Self Attention / Flash Attention

![image.png](explainer_screenshots/flash_attention.png)

Causal self-attention is self-attention with a strictly lower-triangular mask, so each position $i$ can attend only to positions $j≤i$ (no look-ahead). For each position in a sequence, the model decides which earlier tokens matter and blends their information to generate the upcoming token, but the model is not allowed to peek at tokens to the right. Recall that we created an input `x` that has all of the tokens in the sequence. With causal self attention we ensure that the prediction of `y` is not learned in an example from tokens to the right of it. 

Causal self attention uses 3 linear projections to create representations of 3 concepts to help derive the next token: query, key, and value. FlashAttention takes the input `x` of shape $(B,T,C)$, linearly projects the last dimension to `Q, K, V` by multiplying each token vector $x_{b,t,:}$ with learned weight matrices `Q, K,V` (often via one combined QKV linear) and adding biases (which we do not have. Then we compute the causal dot-product attention with softmax per head, concatenate the heads, and apply a final linear layer to produce the output.

The Q,K,V matrix do the following: 
* **Query** - the current position’s “search request.” Every layer and head issues queries that can look for many things (antecedent, rhythm, agreement, etc.).
* **Key** - a matching tag/address for each allowed position. It is compared with the query to produce relevance scores
* **Value** - the payload you actually mix in once something matches. It’s a learned projection of the token’s representation so the model can copy the right kind of information.

This layer starts by taking a linear layer that has `n_embd` rows and then  `n_embd` columns for each: the query, key and value.  This allows the projection of the `x_norm` to be split into the 3 components, selected from them. We then use `scaled_dot_product_attention` to provide the attention masking and normalization, including a lower triangle mask to prevent look-ahead, and final project that onto a `n_embd x n_embd` matrix to return the output. 

We start by creating the 2 default layers. Also we'll set heads to 2.  The heads allow the layer to specialize in different concepts since each head creates its own Q,K,V

*for the linear layers we'll use a similar initiation to the initial embedding layer, just scaled down*

In [ ]:
from torch.nn import functional as F
n_head = 2

In [ ]:
c_attn = nn.Linear(n_embd, 3 * n_embd)
with torch.no_grad():
    out, inp = c_attn.weight.shape  # (3*n_embd, n_embd)
    i = torch.arange(out).unsqueeze(1)  # [out,1]
    j = torch.arange(inp).unsqueeze(0)  # [1,inp]
    c_attn.weight.copy_(0.001 * (1 + i + j))  # W[i,j] = 0.001*(1+i+j)
    c_attn.bias.zero_()

c_attn, c_attn.weight

In [ ]:
c_proj = nn.Linear(n_embd, n_embd)
c_proj

#### Creating Query, Key, Value
We'll now create the query, key, and value.  We'll dynamically derive the size to add scalability. 

In [ ]:
B, T, C = x_norm.size()
B, T, C

In [ ]:
x_norm

Now we'll take x_norm and every projection `C` is added with the corresponding layer in `c_attn` to creation the `qkv` combined matrix leading to a 3x size increase. the `C` in `x_norm` is projected 3 times to align with the 3x `C` dimension in `c_attn`

In [ ]:
qkv = c_attn(x_norm)
qkv.shape, qkv

now, we will split up qkv to create the 3 separate matricies that can ename the query, key, and value to work together to create complex concept embeddings. We also then have to split up each matrix into its own heads (shown as columns `dim=3`) so that concepts can get separately embeded and adjusted during back_prop. 

In [ ]:
q,k,v = qkv.split(n_embd, dim=2)
q = q.view(B, T, n_head, C // n_head).transpose(1, 2)
k = k.view(B, T, n_head, C // n_head).transpose(1, 2)
v = v.view(B, T, n_head, C // n_head).transpose(1, 2)
q.shape, q, k.shape, k, v.shape, v

#### Cross Attention / Dot-product attention. 
With the separated query, key, value, we now need to run the dot product, but restrict each position to only be able to look at embeddings for tokens that have come before it.  To do this, flash attention tiles Q,K,V and applies masking and softmax on the fly. If done without flash attention, you'd need to materializes the large `(T,T)` matrix for all the queries and keys. It then applies a mask fill to zero out the upper triangle, then does a dot product and softmax with the value matrix.  Finally we collapse the heads together back into the same dimension. 

*Note we use `contiguous` here to force `transpose()` to create a new matrix in memory. This allows the heads to learn independently

In [ ]:
fa = F.scaled_dot_product_attention(q, k, v, is_causal=True)
fa = fa.transpose(1, 2).contiguous().view(B, T, C)
fa.shape, fa

#### Final projection and output
Finally, we will now project the cross attention matrix on another final linear layer `n_embd x n_embd`

In [ ]:
x_norm = c_proj(fa)
x_norm.shape, x_norm

#### Transformer - Residual (skip) connection
![image.png](explainer_screenshots/skip_layer.png)

Modern networks also use skip connections, meaning they allows for pathway to bypass around "boxes", passing through through gradients during the backward pass.  This attribute ensures that the impact of each layer and head is normalized against the input embeddings themselves. Recall in the diagram we had the arrow that bypassed "masked multiheaded attention".  Functionally this is represented as

$y = f(x) + x$

To achieve this we simply sum the project matrix `x` with the normalized output `x_norm`.

In [ ]:
x = x + x_norm
x.shape, x

### Layer Normalization 2
We'll run another round of norma now on the outputs of the Masked Multi-head attention and feed forward to ensure our values are not too spread apart.  This layer will run the same  the same normalization formula as before, but is it's own independent layer as it has different inputs. Recall the formula is: 

$y = \frac{x - \mathbb{E}[x]}{\sqrt{\operatorname{Var}[x] + \epsilon}}\,\gamma + \beta$

Because we will do feed forward again for the next layer, MLP, we'll once again branch `x` for the normalization and MLP and then sum it back togehter with `x`

In [ ]:
ln_2 = nn.LayerNorm(n_embd)

In [ ]:
x_norm_2 = ln_2(x)
x_norm_2.shape, x_norm_2

#### Transformer - Feed Forward (aka Multi-layer Perceptron)

![image.png](explainer_screenshots/mlp.png)

The feed-forward sublayer consists of two-layer mirroring a multi-layer perceptron, MLP.  These layers mix the features within each token vector but never across time. The output `x` from multi-headed attention starts as `B x T x C`.  Feed forward 
1. Calculates `4C` using a $XW^T + B$ linear layer 
2. Normalizes the data using a `tanh` based GELU layer. This layer pushes extreme values to +/- 1
3. Projects back down to `C` with a final $XW^T + B$ linear layer.

The MLP nonlinearly re-expresses each token's channel features before being aggregated across the hidden layers and passed to the output layer. 

We'll first start by creating the 3 different layers:
1. `mlp_fc` - Linear layer to project up to `4C`
2. `mlp_gelu` - tanh layer
3. `mlp_proj` - Linear layer to project down to `C`

In [ ]:
mlp_fc = nn.Linear(n_embd, 4 * n_embd)
mlp_gelu = nn.GELU(approximate='tanh')
mlp_proj = nn.Linear(4 * n_embd, n_embd)


mlp_fc, mlp_fc.weight.shape, mlp_gelu, mlp_proj, mlp_proj.weight.shape

Now we'll take `x_norm_2` and every projection `C` is added with the corresponding weights in `mlp_fc` to creation our linear project up to `4c`

In [ ]:
x_mlp = mlp_fc(x_norm_2)
x_mlp.shape, x_mlp

Now we'll apply the tanh approximation (GELU - tanh) which smoothly gates each input `x_mlp`.  This shrinks negative/small values and passing large positives nearly linearly for stable, well-behaved activations.  `tanh` is applied element wise across the full `x_mlp`

In [ ]:
x_mlp = mlp_gelu(x_mlp)
x_mlp.shape, x_mlp

Finally we'll take `x_mlp` and project `4C` back down to `C` using the weights in `mlp_proj`.  Even though the layer weights are `4x16` the transpose in $XW^T + B$ allows to project back down.

In [ ]:
x_mlp = mlp_proj(x_mlp)
x_mlp.shape, x_mlp

#### Transformer - Residual (skip) connection 2

Our transformer also utilizes use skip connections for the feed forward block for passing through gradients during the backward pass.  Recall that  this is represented as

$y = f(x) + x$

We will again simply sum the project matrix `x` with the normalized output `x_mlp`.

In [ ]:
x = x_mlp + x

x.shape, x

#### Transformer - Final Layer Normalization 

The final step in the transformer is to aggregate and normalize before calculating the final projections. This layer is similar to the previous normalization layers. This layer will run the same normalization formula as before, but is it's own independent layer as it has different inputs. Recall the formula is: 

$y = \frac{x - \mathbb{E}[x]}{\sqrt{\operatorname{Var}[x] + \epsilon}}$

Since this is the final layer, we will not have a residual connection so we do not need to branch `x`. 

In [ ]:
ln_f = nn.LayerNorm(n_embd)
ln_f

In [ ]:
x = ln_f(x)
x.shape, x

### Output Layers AKA Model Head.

The combination of masked multi-head attention and feed forward, along with the normalization and residual connections is considered the "transformer".  In practice this layer is horizontally scaled to run many layers in parallel.  Once those layers are complete during the forward pass we then start the output process that results in `logits` which is a representation of the probability of each token being the next token given the input.  

This layer is also known as the model **head**, not to be confused with attention heads. This layer is called this because it is a small, task-specific module attached to a model’s shared backbone that maps hidden features to the final outputs.  In our examples case, this is a linear layer mapping the backbone to vocab logits. The benefit of this structure is that you can use the shared hidden features and train different heads for different tasks without starting from scratch. An example would be a classifier head, or policy head in RL.

![image.png](explainer_screenshots/output_layer.png)

For our head we want to map to to a predicted token which we'll look at as `logits`. In the process to generate `logits` we take the normalize output `x` of the transformers, then project, using a linear layer, to the vocabulary resulting in a `B, T, vocab_size` matrix known as `logits`.  

In training, the `logits` are then compared with `y` to see how close the  model is to predicting the correct next token. For inference, the `logits` are then used to drive sampling which is how the next token is then derived. 


We'll start with initializing a linear projection layer.

In [ ]:
lm_head = nn.Linear(n_embd, vocab_size, bias=False)
lm_head, lm_head.weight

#### Output layer - LM Head aka logits
finally we now project `x` onto the vocabulary resulting in a `B X T X vocab_size` final array `logits`.  This output correlates with the 
probabilty of each output token given the input context.  The best way to read  this is:

(dimension 0) we have 2 batches B, 
(dimension 1) each batch has an example for each value between 1 and context length T 
(dimension 2) for each example we see the probability of each token in our vocabulary

In [ ]:
logits = lm_head(x)

logits.shape, logits

### Loss calculation
Now we have to see how good our prediction is.  Since we haven't done training, we can expect it's pretty shitty. That said, we need to know how much. For this example we'll use cross entropy, also known as the negative log likelihood of the softmax.  Our loss calculates

$$
\ell_i=-\log\big(\mathrm{softmax}(z_i)\_{y_i}\big)
= -z_{i,y_i}+\log\!\sum_{c=1}^C e^{z_{i,c}},
$$


To calculate loss we'll pass in the calculated `logits` and our next tokens stored in `y`. The cross entropy function does not respect batches so we'll flatten the `B` dimension for both `logits` and `y`

In [ ]:
y_flat = y.view(-1)
y_flat.shape, y_flat

In [ ]:
logits_flat = logits.view(-1, logits.size(-1))
logits_flat.shape, logits_flat

In [ ]:
loss = F.cross_entropy(logits_flat, y_flat)
loss.shape, loss